# Laboratorio 4: Clasificación de imagenes
* Javier Jo
* Eric Mendoza
* Marlon Fuentes
___

## Preparación del entorno de ejecución

### Carga de librerías

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import cohen_kappa_score

In [ ]:
# Función para calcular el coeficiene utilizado de comparación
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device='cuda:0')

### Carga de data
Se tomó como ejemplo de carga de data el método utilizado en este [notebook](https://www.kaggle.com/carlolepelaars/efficientnetb5-with-keras-aptos-2019) de Carlo Lepelaars y por eso e forkeó al inicio.

In [ ]:
import os
os.listdir('../input')


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 999
seed_everything(SEED)

In [ ]:
base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df = df.drop(columns=['id_code'])
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df.head

## 1. Análisis exploratorio

### Tamaño de dataset
Se cuenta con la siguiente cantidad de imagenes para entrenar los modelos.

In [ ]:
len_df = len(df)
print(len_df)

### Distribución de variable respuesta
Como se observa en la siguiente gráfica, se tiene una gran cantidad de imagenes con diagnóstico 0. Esto no es beneficioso para realizar las pruebas, sin embargo, por consejo de la profesora Lynette, se considera utilizar los datasets de otros concursos de Kaggle.

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
sns.countplot(df['diagnosis'])
plt.show()

### Dimensiones de imagen
Para poder análizar las imagenes en un tiempo considerable y de manera significativa, la imagen a procesaro no debe estar como nos es entregada. Por ejemplo, la siguiente imagen tiene dimensiones demasiado grandes.

In [ ]:
from PIL import Image
im = Image.open(df['path'][1])
plt.imshow(np.asarray(im))

In [ ]:
width, height = im.size
print("Dimensiones %s, %s" % (width,height)) 

### Redimensionamiento de dataset
Se desea que todas las imagenes tengan la misma cantidad de luz, estén con la misma orientación y tengan el mismo zoom. Se aplicarán las siguientes transformaciones.

In [ ]:
batch = 64
dimension = 224
trans = get_transforms(do_flip=True, flip_vert=True, max_rotate=360, max_warp=0, max_zoom=1.1, max_lighting=0.1, p_lighting=0.5)
src = (ImageList.from_df(df=df,path='./',cols='path').split_by_rand_pct(0.2).label_from_df(cols='diagnosis', label_cls=FloatList))
data = (src.transform(trans, size=dimension, resize_method=ResizeMethod.SQUISH, padding_mode='zeros').databunch(bs=batch, num_workers=4).normalize(imagenet_stats))

## 2. Modelo de redes neuronales simples
Ahora que se tiene normalizado el dataset, se utilizará un modelo de redes neuronales simple, en el que se utilizará la librería fastai.

In [ ]:
simple_model = cnn_learner(data, base_arch=models.resnet50, metrics = [quadratic_kappa])

Se quiere utilizar los parámetros óptimos para la red neuronal, por lo cual se procede a buscar el *learning rate* que minimice el *loss*.

In [ ]:
simple_model.lr_find()
simple_model.recorder.plot(suggestion=True)

Como se observa en la gráfica anterior, el *loss* se minimiza alrededor de 1e-02, por lo que se utilizará ese valor para el learning rate en el entrenamiento.

In [ ]:
simple_model.fit_one_cycle(4,max_lr = 1e-2)

## 3. Modelo de deep learning

## 4. Comparación de algoritmos
Para comparar los modelos se utilizará el Cohen's quadratically weighted kappa, que es el que se utiliza en la competencia de Kaggle para ver qué tan bueno es el modelo. Por lo tanto, procedemos a mostrar los coeficientes obtenidos para cada uno.

### Modelo neuronal simple

### Deep Learning

### Conclusión
Debido a la 